In [ ]:
import subprocess
import numpy as np
import pandas as pd
import re
import astropy.units as u
import gala.dynamics as gala_dynamics
import os
from gala.units import galactic
import plotly.express as px
from gc_stream_toolkit import read_nemo

In [ ]:

os.environ['NEMO_PATH'] = '/Users/iraevetts/CLionProjects/FalCone/nemo'

filename = r"/Users/iraevetts/CLionProjects/FalCone/nemo/ngc6569_realistic_mw.dat"


# Get the last timestep
data = read_nemo(filename, timestep=-1)
print(f"Time: {data.time}, Particles: {data.particle_count}")

# Or get an earlier timestep
data_early = read_nemo(filename, timestep=0)  # First snapshot
print(f"Time: {data_early.time}")


In [ ]:
nemo_data=data
tidal_tails_interactive_fig = px.scatter_3d(
    x=nemo_data.phase_space_position.x.value,
    y=nemo_data.phase_space_position.y.value,
    z=nemo_data.phase_space_position.z.value,
    title='FALCON N-body Simulation',
    labels={'x': 'X [kpc]', 'y': 'Y [kpc]', 'z': 'Z [kpc]'},
    opacity=.6,
)

tidal_tails_interactive_fig.update_traces(marker=dict(size=1))
tidal_tails_interactive_fig.update_layout(width=800, height=800, scene=dict(aspectmode='cube'))
tidal_tails_interactive_fig.show()

In [ ]:
import astropy.units as u
import astropy.coordinates as coord

# Your observational data (heliocentric)
ngc6569_helio = coord.SkyCoord(
    ra=273.412 * u.degree,
    dec=-31.827 * u.degree,
    distance=10.53 * u.kpc,
    pm_ra_cosdec=-4.125 * u.mas/u.yr,
    pm_dec=-7.354 * u.mas/u.yr,
    radial_velocity=-49.82 * u.km/u.s,
    frame='icrs'
)

print("Heliocentric coordinates:")
print(f"RA: {ngc6569_helio.ra}")
print(f"Dec: {ngc6569_helio.dec}")
print(f"Distance: {ngc6569_helio.distance}")
print(f"PM RA: {ngc6569_helio.pm_ra_cosdec}")
print(f"PM Dec: {ngc6569_helio.pm_dec}")
print(f"RV: {ngc6569_helio.radial_velocity}")

# Convert to galactocentric frame
ngc6569_galactocentric = ngc6569_helio.transform_to(coord.Galactocentric())

print(f"\nGalactocentric coordinates:")
print(f"X: {ngc6569_galactocentric.x:.3f}")
print(f"Y: {ngc6569_galactocentric.y:.3f}")
print(f"Z: {ngc6569_galactocentric.z:.3f}")
print(f"v_x: {ngc6569_galactocentric.v_x:.3f}")
print(f"v_y: {ngc6569_galactocentric.v_y:.3f}")
print(f"v_z: {ngc6569_galactocentric.v_z:.3f}")

# Convert velocities to NEMO units (kpc/Myr)
vx_nemo = ngc6569_galactocentric.v_x.to(u.kpc/u.Myr)
vy_nemo = ngc6569_galactocentric.v_y.to(u.kpc/u.Myr)
vz_nemo = ngc6569_galactocentric.v_z.to(u.kpc/u.Myr)

print(f"\nFor NEMO simulation:")
print(f"Position: ({ngc6569_galactocentric.x.value:.3f}, {ngc6569_galactocentric.y.value:.3f}, {ngc6569_galactocentric.z.value:.3f}) kpc")
print(f"Velocity: ({vx_nemo.value:.1f}, {vy_nemo.value:.1f}, {vz_nemo.value:.1f}) kpc/Myr")

In [ ]:
from gc_stream_toolkit import get_cluster
import numpy as np

cluster = get_cluster('ngc6569')
gc_coords = cluster.galactocentric

# Print coordinates in NEMO's 10 kpc units
print(f'X: {gc_coords.x.value/10:.3f}')
print(f'Y: {gc_coords.y.value/10:.3f}')
print(f'Z: {gc_coords.z.value/10:.3f}')
print(f'VX: {gc_coords.v_x.value/100:.3f}')
print(f'VY: {gc_coords.v_y.value/100:.3f}')
print(f'VZ: {gc_coords.v_z.value/100:.3f}')

In [ ]:
import os

# Set the NEMO path for your Python reader
os.environ['NEMO_PATH'] = '/Users/iraevetts/CLionProjects/FalCone/nemo'

# Also set NEMO for the tsf command
os.environ['NEMO'] = '/Users/iraevetts/CLionProjects/FalCone/nemo'

# Change to the correct directory
os.chdir('/Users/iraevetts/CLionProjects/FalCone/nemo')

# Now try the reader
from gc_stream_toolkit import read_nemo
data_start = read_nemo("orbit_reasonable.dat", timestep=0)
print(f"Success! Positions: {data_start.positions}")

In [ ]:
import subprocess
import os

def count_snapshots(filename):
    """Count number of snapshots in NEMO file"""
    nemo_path = os.environ['NEMO_PATH']
    tsf_path = os.path.join(nemo_path, 'bin', 'tsf')

    result = subprocess.run([tsf_path, filename],
                          capture_output=True, text=True)

    # Count occurrences of "set SnapShot"
    snapshot_count = result.stdout.count('set SnapShot')
    return snapshot_count

# Test it
num_snapshots = count_snapshots("long_orbit.dat")
print(f"File contains {num_snapshots} snapshots")

In [ ]:
def plot_full_orbits_3d(filename, particle_indices=None):
    """Plot orbits in 3D"""

    # Use the same data reading logic
    num_snapshots = count_snapshots(filename)
    all_positions = []
    all_times = []

    for i in range(num_snapshots):
        data = read_nemo(filename, timestep=i)
        all_positions.append(data.positions)
        all_times.append(data.time)

    positions = np.array(all_positions)
    times = np.array(all_times)

    # 3D plot
    from mpl_toolkits.mplot3d import Axes3D

    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111, projection='3d')

    num_particles = positions.shape[1]
    if particle_indices is None:
        particle_indices = range(num_particles)

    colors = ['blue', 'red', 'green', 'orange', 'purple', 'brown']

    for i in particle_indices:
        if i < num_particles:
            x_traj = positions[:, i, 0]
            y_traj = positions[:, i, 1]
            z_traj = positions[:, i, 2]

            ax.plot(x_traj, y_traj, z_traj, '-',
                   color=colors[i % len(colors)], alpha=0.8, linewidth=2,
                   label=f'Particle {i+1}')

            # Start and end points
            ax.scatter(x_traj[0], y_traj[0], z_traj[0],
                      color=colors[i % len(colors)], s=100, marker='o')
            ax.scatter(x_traj[-1], y_traj[-1], z_traj[-1],
                      color=colors[i % len(colors)], s=80, marker='s')

    ax.set_xlabel('X [NEMO units]')
    ax.set_ylabel('Y [NEMO units]')
    ax.set_zlabel('Z [NEMO units]')
    ax.set_title(f'3D Orbital Trajectories\nTime: {times[0]:.2f} to {times[-1]:.2f}')
    ax.legend()

    plt.show()

# Try both 2D and 3D
plot_full_orbits_3d("long_orbit.dat")